# Предсказание корзины пользователя поиском похожих заказов в истории

Суть подхода:  
    1. Нахожу последний заказ пользователя  
    2. Нахожу точно такие же или похожие заказы в истории  
    3. Предсказываю тот заказ, который идет после такого же (или похожих)  

Модель сходства:
    SIM = len(A & B) ** 2 / (len(A) * len(B))
    Сходство = длина множества пересечений 2-х заказов в квадрате, деленная на произведение длин заказов.
    Например: 2 заказа, в каждом 5 полностью одинаковых позиций.
    SIM = 5 ** 2 / (5 * 5) = 1 - максимальное сходство
    
    Например: 2 заказа, в одном 5 позиций, в другом - 7 позиций, 4 из них общие.
    SIM = 4 ** 2 / (5 * 7) = 0,4571 - высокое сходство
    
    Например: 2 заказа, в одном 3 позиций, в другом - 20 позиций, 2 из них общие.
    SIM = 2 ** 2 / (3 * 20) = 0,0667 - низкое сходство

Есть альтернативное сходство:
    SIM = len(A & B) / (len(A | B))
    Сходство = длина множества пересечений 2-х заказов, деленная на длину пересечения заказов.
    Например: 2 заказа, в каждом 5 полностью одинаковых позиций.
    SIM = 5 / 5 = 1 - максимальное сходство
    
    Например: 2 заказа, в одном 5 позиций, в другом - 7 позиций, 4 из них общие.
    SIM = 4 / 8 = 0,5 - высокое сходство
    
    Например: 2 заказа, в одном 3 позиций, в другом - 20 позиций, 2 из них общие.
    SIM = 2 / 21 = 0,0952 - низкое сходство
    
Данная модель работает примерно также, но в целом значения сходства получаются выше.

In [152]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
df = pd.read_csv('train.csv')

In [4]:
df['order_completed_at'] = pd.to_datetime(df['order_completed_at'])  # Привожу к дате-времени

In [5]:
df.head(3)

,user_id,order_completed_at,cart
0,2,2015-03-22 09:25:46,399
1,2,2015-03-22 09:25:46,14
2,2,2015-03-22 09:25:46,198


In [6]:
# Выходной файл
sample_out_df = pd.read_csv('sample_submission.csv')

In [7]:
sample_out_df.head(3)

,id,target
0,0;133,0
1,0;5,1
2,0;10,0


# Подготовка данных

In [8]:
# Выделяю пользователя и товар в отдельные колонки
sample_out_df['user_category'] = sample_out_df['id'].apply(lambda x: x.split(';'))
sample_out_df['user'] = sample_out_df['user_category'].apply(lambda x: int(x[0]))
sample_out_df['category'] = sample_out_df['user_category'].apply(lambda x: int(x[1]))

In [9]:
sample_out_df.head(3)

,id,target,user_category,user,category
0,0;133,0,"[0, 133]",0,133
1,0;5,1,"[0, 5]",0,5
2,0;10,0,"[0, 10]",0,10


In [10]:
out_categories = pd.unique(sample_out_df['category'])
out_categories = np.sort(out_categories).tolist()
print('Количество уникальных категорий:', len(out_categories))

Количество уникальных категорий: 858


In [11]:
# Удаляю из основного фрейма все категории, которые не требуется предсказывать
# т.е. тех, которых нет в out_categories
df_cut = df[df['cart'].isin(out_categories)]

In [12]:
# Формирую датасет с заказами:
orders_df = df_cut.groupby(['user_id', 'order_completed_at'])['cart'].apply(set).to_frame().reset_index()

In [13]:
orders_df.head()

,user_id,order_completed_at,cart
0,0,2020-07-19 09:59:17,"{14, 430, 82, 20, 405, 441, 379, 57}"
1,0,2020-08-24 08:55:32,"{5, 133, 10, 396, 14, 402, 405, 22, 409, 25, 2..."
2,0,2020-09-02 07:38:25,"{803, 169, 170, 398, 399, 401, 84, 55, 440, 57..."
3,1,2019-05-08 16:09:41,{55}
4,1,2020-01-17 14:44:23,"{421, 204, 82, 86, 55, 798}"


In [14]:
out_users = pd.unique(sample_out_df['user'])
out_users = np.sort(out_users).tolist()
print('Количество уникальных пользователей для предсказания:', len(out_users))

Количество уникальных пользователей для предсказания: 13036


In [15]:
all_users = pd.unique(df['user_id'])
all_users = np.sort(all_users).tolist()
print('Всего уникальных пользователей:', len(all_users))

Всего уникальных пользователей: 20000


In [16]:
# Разделяю последние заказы каждого пользователя и остальные
last_orders_df = pd.DataFrame()
not_last_orders_df = pd.DataFrame()

for user in all_users:
    sorted_order = orders_df[orders_df['user_id'] == user]  \
    .sort_values(by='order_completed_at', ascending=False)  \
    .reset_index(drop=True)
    
    last_orders_df = pd.concat([last_orders_df, sorted_order.loc[0:0, :]])
    not_last_orders_df = pd.concat([not_last_orders_df, sorted_order.loc[1:, :]])

In [17]:
# Сброс индексов
last_orders_df = last_orders_df.reset_index(drop=True)
not_last_orders_df = not_last_orders_df.reset_index(drop=True)

In [19]:
# Подсчет длин непоследних заказов
not_last_orders_df.loc[:, 'len'] = not_last_orders_df['cart'].apply(lambda x: len(x))
not_last_orders_df

,user_id,order_completed_at,cart,len
0,0,2020-08-24 08:55:32,"{5, 133, 10, 396, 14, 402, 405, 22, 409, 25, 2...",25
1,0,2020-07-19 09:59:17,"{14, 430, 82, 20, 405, 441, 379, 57}",8
2,1,2020-05-24 11:13:59,"{169, 170, 171, 14, 55, 798}",6
3,1,2020-04-30 17:45:22,{55},1
4,1,2020-04-14 01:31:20,"{812, 798, 55}",3
...,...,...,...,...
189399,19997,2020-08-30 11:23:41,"{398, 14, 17, 23, 49, 55, 57, 443, 61, 197, 19...",24
189400,19998,2020-09-01 08:12:32,"{420, 6, 398, 84, 57, 61, 415}",7
189401,19998,2020-08-30 12:15:55,"{420, 6, 398, 57, 26, 31, 29, 415}",8
189402,19999,2020-08-31 19:32:08,{326},1


In [20]:
# Максимальная длина заказа (в товарных позициях)
max_len_order = max(not_last_orders_df['len'])
max_len_order

120

# Поиск последнего заказа и похожего на него

In [153]:
best_sims = {}  # Для контроля похожести
predictions = {}  # Предсказания для пользователей

In [154]:
with tqdm(total=len(out_users)) as pbar:
    for user in out_users:
        # Последний заказ текущего пользователя
        last_order = last_orders_df[last_orders_df['user_id'] == user].reset_index(drop=True)
        # Длина последнего заказа текущего пользователя
        last_order_len = len(last_order.loc[0, 'cart'])
        # Проставляю разницу длин заказов
        not_last_orders_df.loc[:, 'len_diff'] = abs(not_last_orders_df['len'] - last_order_len)
        not_last_orders_df = not_last_orders_df.sort_values(by='len_diff').reset_index(drop=True)
        # Максимальная разница между текущим заказом и всеми остальными
        max_len_diff = max(not_last_orders_df['len_diff'])

        # Поиск заказа, максимально похожего на последний.
        # Поиск начинается с заказов, у которых минимальный len_diff
        best_sim = 0  # Текущее лучшее сходство
        best_df = pd.DataFrame()
        for diff in range(max_len_diff + 1):
            ideal_sim = last_order_len / (last_order_len + diff)  # максимальное сходство, которое можно достичь с текущим diff
            if best_sim >= ideal_sim:
                # дальше искать бессмысленно, т.к. достичь лучшего сходства невозможно теоретически
                break
            cur_df = not_last_orders_df[not_last_orders_df['len_diff'] == diff]
            if len(cur_df) > 0:
                cur_df.loc[:, 'sim'] = cur_df['cart']  \
                        .apply(lambda x: len(x & last_order.loc[0, 'cart']) ** 2 / (len(x) * len(last_order.loc[0, 'cart'])))

                if max(cur_df['sim']) > best_sim:
                    best_sim = max(cur_df['sim'])
                    best_df = cur_df[cur_df['sim'] == best_sim]
                elif max(cur_df['sim']) == best_sim:
                    best_df = pd.concat([best_df, cur_df[cur_df['sim'] == best_sim]])
        # Запоминаю лучшее сходство для последнего заказа текущего пользователя
        best_sims[user] = best_sim
        # В лучших заказах оставляю только заказы текущего пользователя (если они есть)
        best2_df = best_df[best_df['user_id'] == user]
        if len(best2_df) > 0:
            best_df = best2_df
        best_df = best_df.reset_index(drop=True)
        # Из лучших заказов беру один случайный заказ и назначаю его ближайшим
        closest_order = best_df.iloc[0]
        # Нахожу заказ, следующий за ближайшим
        next_orders = not_last_orders_df[(not_last_orders_df['user_id'] == closest_order['user_id'])
                                       & (not_last_orders_df['order_completed_at'] > closest_order['order_completed_at'])]  \
                    .sort_values(by='order_completed_at')  \
                    .reset_index(drop=True)
        if len(next_orders) == 0:
            # Значит следующий заказ лежит в last_orders
            next_orders = last_orders_df[last_orders_df['user_id'] == closest_order['user_id']].reset_index(drop=True)
        next_cart = next_orders.loc[0, 'cart']
        # Сохраняю предсказания по текущему пользователю
        predictions[user] = next_cart
        
        pbar.update()

100%|██████████| 13036/13036 [6:51:57<00:00,  1.90s/it]  


In [155]:
predictions

{0: {5, 14, 17, 25, 55, 57, 84, 383, 385, 398, 430, 432, 434, 440},
 1: {54, 55, 84, 112, 149, 157, 169, 256},
 3: {14, 15, 18, 19, 22, 25, 82, 87, 100, 146, 380, 382, 398, 402},
 4: {14, 203, 231, 302, 376, 382},
 5: {5, 11, 14, 15, 17, 23, 54, 55, 61, 86, 99, 100, 105, 112, 396, 402, 409},
 7: {5, 9, 14, 19, 22, 23, 26, 89, 99, 104, 396, 398, 411, 712, 804},
 8: {14, 15, 17, 19, 23, 31, 57, 67, 84, 248, 256, 380, 383, 384, 420},
 9: {9, 14, 17, 19, 23, 29, 425},
 10: {9,
  14,
  18,
  19,
  30,
  42,
  43,
  57,
  61,
  85,
  87,
  100,
  383,
  384,
  388,
  398,
  410,
  411,
  420,
  430,
  431,
  441,
  445,
  804},
 11: {9, 19, 21, 23, 55, 172, 799, 800},
 12: {14,
  19,
  22,
  27,
  32,
  57,
  67,
  84,
  92,
  377,
  379,
  380,
  383,
  385,
  389,
  391,
  395,
  397,
  419},
 13: {29, 41, 57, 84, 109, 140, 157, 169, 173, 383, 411, 430},
 14: {14, 16, 17, 18, 19, 26, 61, 79, 102, 220, 380, 398, 409, 440, 627},
 15: {9,
  10,
  14,
  17,
  18,
  29,
  55,
  57,
  61,
  62,


In [156]:
import pickle

In [157]:
with open('predictions_order_sim.pickle', 'wb') as f:
    pickle.dump(predictions, f, protocol=pickle.HIGHEST_PROTOCOL)

# with open('predictions_order_sim.pickle', 'rb') as f:
#     predictions = pickle.load(f)

In [173]:
with open('best_sims.pickle', 'wb') as f:
    pickle.dump(best_sims, f, protocol=pickle.HIGHEST_PROTOCOL)

# with open('predictions_order_sim.pickle', 'rb') as f:
#     best_sims = pickle.load(f)

In [167]:
# В среднем каждому заказу удается найти похожий со степенью сходства 0.4671
np.array(list(best_sims.values())).mean()

0.4671438254832681

In [166]:
# У половины заказов (медиана) максимально похожий заказ имеет степерь сходства 0.4322
import statistics

items = list(best_sims.values())
statistics.median(items)

0.43218396054073804

# Делаю submission

In [169]:
# Обновления таргетов в соответствиями с предсказаниями нейросети:
sample_out_df['target'] = sample_out_df.apply(lambda x: x['category'] in predictions[x['user']], axis=1).astype(int)

In [170]:
# Еще раз прогноз для пользователя №3
CUR_USER_ID = 3
predictions[CUR_USER_ID]

{14, 15, 18, 19, 22, 25, 82, 87, 100, 146, 380, 382, 398, 402}

In [171]:
# Проверяю глазами: если категория есть в предсказании, то 1. Иначе - 0
# 398 и 16 категории присутствуют - для них 1 в target - все верно.
sample_out_df[sample_out_df['user'] == CUR_USER_ID].head()

,id,target,user_category,user,category
56,3;134,0,"[3, 134]",3,134
57,3;398,1,"[3, 398]",3,398
58,3;399,0,"[3, 399]",3,399
59,3;15,1,"[3, 15]",3,15
60,3;16,0,"[3, 16]",3,16


In [172]:
sub_df = sample_out_df.copy()
sub_df = sub_df.loc[:, ['id', 'target']]
sub_df.to_csv('sub51.csv', index=False)   # Score = 0.31554

# Выводы:

Т.к. заказы в целом не похожи друг на друга (в среднем удается найти заказ со степенбю сходства только 0.4322),  
то значит и следующий заказ будет не сильно похожим. Score об этом красноречиво говорит.
Но можно попробовать взять заказы, которые похожи, например, на 0.95 (этот порог можно подвигать),  
а в остальных случаях делать предсказания на основе лучшей имеющейся на текущий момент модели (LSTM)

# Часть 2

Пробую совместить лучшую модель LSTM с предсказаниями по заказам с высокой степенью сходства

In [174]:
# Всего предсказаний для пользователей
len(best_sims)

13036

In [274]:
top_sims = {}
THRESHOLD = 0.85

for user, sim in best_sims.items():
    if sim >= THRESHOLD:
        top_sims[user] = sim

In [275]:
len(top_sims)

616

In [264]:
#Загружаю лучшее предсказание LSTM
sample_out_df = pd.read_csv('sub45.csv')
sample_out_df['user_category'] = sample_out_df['id'].apply(lambda x: x.split(';'))
sample_out_df['user'] = sample_out_df['user_category'].apply(lambda x: int(x[0]))
sample_out_df['category'] = sample_out_df['user_category'].apply(lambda x: int(x[1]))
sample_out_df.head(3)

,id,target,user_category,user,category
0,0;133,0,"[0, 133]",0,133
1,0;5,0,"[0, 5]",0,5
2,0;10,0,"[0, 10]",0,10


In [265]:
sample_out_df2 = sample_out_df.copy()

In [266]:
for user in top_sims:
    sample_out_df2.loc[sample_out_df2['user'] == user, 'target']  \
        = sample_out_df2.loc[sample_out_df2['user'] == user].apply(lambda x: x['category'] in predictions[x['user']], axis=1).astype(int)

In [267]:
sample_out_df[sample_out_df['user'] == 86]

,id,target,user_category,user,category
5634,86;0,0,"[86, 0]",86,0
5635,86;5,0,"[86, 5]",86,5
5636,86;6,0,"[86, 6]",86,6
5637,86;9,1,"[86, 9]",86,9
5638,86;10,1,"[86, 10]",86,10
...,...,...,...,...,...
5753,86;437,0,"[86, 437]",86,437
5754,86;440,0,"[86, 440]",86,440
5755,86;443,0,"[86, 443]",86,443
5756,86;445,0,"[86, 445]",86,445


In [268]:
sample_out_df2[sample_out_df2['user'] == 86]

,id,target,user_category,user,category
5634,86;0,1,"[86, 0]",86,0
5635,86;5,0,"[86, 5]",86,5
5636,86;6,0,"[86, 6]",86,6
5637,86;9,0,"[86, 9]",86,9
5638,86;10,0,"[86, 10]",86,10
...,...,...,...,...,...
5753,86;437,0,"[86, 437]",86,437
5754,86;440,0,"[86, 440]",86,440
5755,86;443,0,"[86, 443]",86,443
5756,86;445,0,"[86, 445]",86,445


In [269]:
predictions[86]

{0, 379, 380}

In [270]:
top_sims

{26: 1.0,
 86: 1.0,
 148: 1.0,
 149: 1.0,
 158: 1.0,
 178: 1.0,
 195: 1.0,
 234: 0.8888888888888888,
 240: 1.0,
 247: 1.0,
 267: 1.0,
 274: 1.0,
 277: 1.0,
 289: 1.0,
 315: 1.0,
 374: 1.0,
 411: 1.0,
 421: 1.0,
 435: 1.0,
 448: 1.0,
 496: 1.0,
 532: 1.0,
 573: 1.0,
 661: 1.0,
 683: 1.0,
 723: 1.0,
 770: 1.0,
 837: 1.0,
 839: 1.0,
 854: 1.0,
 988: 1.0,
 1042: 1.0,
 1050: 1.0,
 1084: 0.8888888888888888,
 1132: 1.0,
 1147: 1.0,
 1156: 1.0,
 1174: 1.0,
 1186: 1.0,
 1255: 1.0,
 1271: 1.0,
 1292: 1.0,
 1295: 0.8928571428571429,
 1300: 1.0,
 1324: 1.0,
 1344: 1.0,
 1346: 1.0,
 1350: 1.0,
 1378: 1.0,
 1532: 1.0,
 1557: 1.0,
 1650: 1.0,
 1707: 1.0,
 1727: 0.8888888888888888,
 1767: 1.0,
 1775: 1.0,
 1822: 1.0,
 1826: 1.0,
 1848: 1.0,
 1857: 1.0,
 1863: 1.0,
 1878: 1.0,
 1947: 1.0,
 1948: 1.0,
 1998: 0.875,
 2047: 1.0,
 2076: 1.0,
 2087: 1.0,
 2097: 1.0,
 2121: 1.0,
 2124: 1.0,
 2152: 1.0,
 2168: 1.0,
 2189: 1.0,
 2190: 1.0,
 2203: 1.0,
 2234: 1.0,
 2251: 1.0,
 2366: 1.0,
 2368: 1.0,
 2370: 1.0,

In [271]:
sub_df = sample_out_df2.copy()
sub_df = sub_df.loc[:, ['id', 'target']]
sub_df.to_csv('sub53.csv', index=False)   # Score = 0.47801
# Результат немного ухудшился (с 0.48032)